In [1]:
# Import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import calendar
import datetime
from datetime import datetime, timedelta, date
import warnings
import os

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import ensemble
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score
#settings
pd.set_option('display.max_row', 100)
pd.set_option('display.max_columns', 1200)
pd.set_option('display.width', 1200)
warnings.filterwarnings("ignore")
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
#variables
colour=['maroon','r','g','darkgreen','c','teal','b','navy','indigo','m','deeppink','orange','sienna','yellow','khaki','olive','tan','black','grey','brown']

Packages installed


In [2]:
# import all files in Data folder and read into dataframes
dataframes=[]
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        file=filename.split('.')
        file=((file[0]+"_df"))
        if file !="_df":
            filepath=os.path.join(dirname,filename)
            df=pd.read_csv(filepath,sep=",",encoding = "UTF-8")
            exec(f'{file} = df.copy()')
            print(file, df.shape)
            dataframes.append(df)
print('Data imported')

appearances_df (1175579, 13)
clubs_df (411, 16)
club_games_df (124494, 11)
competitions_df (43, 10)
games_df (62247, 21)
game_events_df (568252, 7)
players_df (28472, 23)
player_valuations_df (424867, 9)
Data imported


In [3]:
players_df_new = players_df.copy()

# Calculate the age of each player
players_df_new['date_of_birth'] = pd.to_datetime(players_df['date_of_birth'])

print(players_df_new['date_of_birth'])
# drop players with no date of birth
#players_df_new = players_df[players_df['date_of_birth'].isnull() == False]

# Count how many rows have missing date_of_birth values
null_date_of_birth_count = players_df_new['date_of_birth'].isnull().sum()

print("Number of rows with missing date_of_birth:", null_date_of_birth_count)

#fill missing values
default_date = pd.to_datetime('2000-01-01')  # Replace '2000-01-01' with your desired default date
players_df_new['date_of_birth'].fillna(default_date, inplace=True)

now = datetime.now()
players_df_new['age'] = (now - players_df_new['date_of_birth']).apply(lambda x: x.days) / 365.25
players_df_new['age'] = players_df_new['age'].round().astype(int)

0       1981-05-01
1       1983-11-05
2       1981-02-18
3              NaT
4       1983-08-14
           ...    
28467   1990-02-28
28468   1991-10-26
28469   1992-12-22
28470   1994-04-20
28471   1999-09-22
Name: date_of_birth, Length: 28472, dtype: datetime64[ns]
Number of rows with missing date_of_birth: 42


In [4]:
players_df_new_backup = players_df_new.copy()
# Calculate the contract remaining of each player
players_df_new['contract_expiration_date'] = pd.to_datetime(players_df_new['contract_expiration_date'])

# Count how many rows have missing date_of_birth values
null_date_of_contract_expiration_date = players_df_new['contract_expiration_date'].isnull().sum()

print("Number of rows with missing contract_expiration_date:", null_date_of_contract_expiration_date)

players_df_new = players_df_new.drop('contract_expiration_date', axis=1)

Number of rows with missing contract_expiration_date: 10509


In [5]:
# add year to player valuations
player_valuations_df['datetime']=pd.to_datetime(player_valuations_df['datetime'], format="%Y-%m-%d")
player_valuations_df['year']=player_valuations_df['datetime'].dt.year

# add year to player appearances
appearances_df['datetime']=pd.to_datetime(appearances_df['date'], format="%Y-%m-%d")
appearances_df['year']=appearances_df['datetime'].dt.year

In [6]:
#add position to player valuations
position_df=players_df_new.copy()
position_df=position_df.drop(['name', 'current_club_id', 'current_club_name', 'country_of_citizenship', 'country_of_birth', 'city_of_birth', 'date_of_birth', 'foot', 'height_in_cm', 'market_value_in_eur', 'highest_market_value_in_eur', 'agent_name', 'current_club_domestic_competition_id', 'first_name', 'last_name', 'player_code', 'image_url', 'last_season', 'url', 'age'], axis=1)
player_valuations_df=player_valuations_df.merge(position_df, left_on='player_id', right_on='player_id')

#add position to appearances
value_df=players_df_new.copy()
value_df=value_df.drop(['name', 'current_club_id', 'current_club_name', 'country_of_citizenship', 'country_of_birth', 'city_of_birth', 'date_of_birth', 'foot', 'height_in_cm', 'sub_position', 'highest_market_value_in_eur', 'agent_name', 'current_club_domestic_competition_id', 'first_name', 'last_name', 'player_code', 'image_url', 'last_season', 'url', 'age'], axis=1)
appearances_df1=value_df.merge(appearances_df, left_on='player_id', right_on='player_id')

print("Data processing complete")

Data processing complete


In [7]:
players_df = players_df_new.copy()
print(players_df)

       player_id      first_name    last_name                 name  last_season  current_club_id          player_code country_of_birth city_of_birth country_of_citizenship date_of_birth        sub_position    position   foot  height_in_cm  market_value_in_eur  highest_market_value_in_eur       agent_name                                          image_url                                                url current_club_domestic_competition_id      current_club_name  age
0            597       Aleksandr         Hleb       Aleksandr Hleb         2016             2696       aleksandr-hleb            UdSSR         Minsk                Belarus    1981-05-01  Attacking Midfield    Midfield    NaN           NaN                  NaN                   19000000.0  fair-sport GmbH  https://img.a.transfermarkt.technology/portrai...  https://www.transfermarkt.co.uk/aleksandr-hleb...                                  RU1  Krylya Sovetov Samara   42
1           1428            Mike        Hanke         

In [8]:
#filter range
player_valuations_df = player_valuations_df[(player_valuations_df.year > 2004 ) & (player_valuations_df.year < 2023 )]
high_value_player_valuations_df = player_valuations_df[(player_valuations_df.market_value_in_eur > 40000000 )]
positions=players_df.position.unique()

#overview of player dataset
#remove players with no Market Value
players_df1 = players_df[players_df.highest_market_value_in_eur.isnull() == False]
#order by Market Value
players_df1 = players_df1.sort_values("highest_market_value_in_eur", ascending = False)
#filter by season and value
players_df1 = players_df[players_df['last_season'] >= 2021]
players_df1 = players_df1[players_df1['highest_market_value_in_eur'] >= 10000000]
high_value_players_df = players_df1[(players_df1.market_value_in_eur > 40000000 )]
positions=players_df.position.unique()

In [9]:
#filter range
appearances_df = appearances_df[(appearances_df.year > 2004 ) & (appearances_df.year < 2023 )]
high_value_appearances_df = appearances_df1[(appearances_df1.market_value_in_eur > 40000000 )]

#add year to game valuations
games_df['datetime']=pd.to_datetime(games_df['date'], format="%Y-%m-%d")
games_df['year']=games_df['datetime'].dt.year

#filter range
games_df = games_df[(games_df.year > 2004 ) & (games_df.year < 2023 )]

In [10]:
# This was by attempt to collate all of the player data, but i found a far more elegant solution from LOIS CORDEIRO see below

#Start with players_df
merged_players_df=players_df.drop(['current_club_id', 'city_of_birth', 'date_of_birth','first_name', 'last_name', 'player_code', 'image_url', 'url'], axis=1)

#next look at clubs_df
merged_players_df = merged_players_df.reindex(columns = merged_players_df.columns.tolist() + ['club_value','squad_size','goals','goals_2022','games_2022','assists_2022','minutes_played_2022','goals_against_2022','goals_for_2022','clean_sheet_2022'])
for player_id in merged_players_df.player_id.unique():
    #print(players_df.current_club_id[(players_df.player_id==player_id)])
    club_id= players_df.current_club_id[(players_df.player_id==player_id)]
    #print(clubs_df.total_market_value[(clubs_df.club_id==int(club_id))])
    #print(clubs_df.squad_size[(clubs_df.club_id==int(club_id))])
    try:
        merged_players_df.club_value[(players_df.player_id==player_id)]=int(clubs_df.total_market_value[(clubs_df.club_id==int(club_id))])
    except:
        merged_players_df.club_value[(players_df.player_id==player_id)]='NaN'  
    merged_players_df.squad_size[(players_df.player_id==player_id)]=int((clubs_df.squad_size[(clubs_df.club_id==int(club_id))]))
#sort column order
columns=['player_id','games_2022','minutes_played_2022','goals_2022','assists_2022','goals_against_2022','goals_for_2022','clean_sheet_2022','name','position','sub_position','last_season','foot','height_in_cm','age','country_of_citizenship','country_of_birth','current_club_name','club_value','squad_size','current_club_domestic_competition_id','agent_name','market_value_in_eur','highest_market_value_in_eur']
merged_players_df=merged_players_df[columns]
#print(merged_players_df.head())

#merge games and appearances
games_and_appearances_df = appearances_df.merge(games_df, on=['game_id'], how='left')
print('Player and club data merged');#print(games_and_appearances_df.shape);print(games_and_appearances_df.columns);print(games_and_appearances_df.head(1))

Player and club data merged


In [11]:
#merge games and appearances   
games_and_appearances_df = appearances_df.merge(games_df, on=['game_id'], how='left')
season = 2022 

#create a function to collate player stats
def player_stats(player_id, season, df):
    df = games_and_appearances_df[games_and_appearances_df['player_id'] == player_id]
    df =  df[ df['season'] == season]    
    if (df.shape[0] == 0):
        Out = [(np.nan, season,0,0,0,0,0,0,0,0,0)]
        out_df = pd.DataFrame(data = Out, columns = ['player_id','season','goals','games',
                                                     'assists','minutes_played','goals_for','goals_against','clean_sheet','yellow_cards','red_cards'])
        return out_df    
    else:       
        df["goals_for"] = df.apply(lambda row: row['home_club_goals'] if row['home_club_id'] == row['player_club_id'] 
            else row['away_club_goals'] if row['away_club_id'] == row['player_club_id'] 
            else np.nan, axis=1)
        df["goals_against"] = df.apply(lambda row: row['away_club_goals'] if row['home_club_id'] == row['player_club_id'] 
            else row['home_club_goals'] if row['away_club_id'] == row['player_club_id'] 
            else np.nan, axis=1)
        df['clean_sheet'] = df.apply(lambda row: 1 if row['goals_against'] == 0
            else 0 if row['goals_against'] > 0
            else np.nan, axis=1)
        df = df.groupby(['player_id',"season"],as_index=False).agg({'goals': 'sum', 'game_id': 'nunique', 
                                                                      'assists': 'sum', 'minutes_played' : 'sum', 'goals_for' : 'sum',
                                                                      'goals_against' : 'sum', 'clean_sheet' : 'sum','yellow_cards':'sum','red_cards':'sum'})
        out_df = df.rename(columns={'game_id': 'games'})
        return out_df
#print(player_stats(67064, 2014, games_and_appearances_df)) #test function works.

#iterate through players
for index in merged_players_df.index:
    id = merged_players_df.loc[index][0]
    #print(id)
    name = merged_players_df.loc[index][1]
    stats = player_stats(id, season, games_and_appearances_df)
    try:
        merged_players_df.at[index,'games_{}'.format(season)]= stats['games'][0]
        merged_players_df.at[index,'goals_{}'.format(season)]= stats['goals'][0]
        merged_players_df.at[index,'assists_{}'.format(season)]= stats['assists'][0]
        merged_players_df.at[index,'minutes_played_{}'.format(season)]= stats['minutes_played'][0]
        merged_players_df.at[index,'goals_for_{}'.format(season)]= stats['goals_for'][0]
        merged_players_df.at[index,'goals_against_{}'.format(season)]= stats['goals_against'][0]
        merged_players_df.at[index,'clean_sheet_{}'.format(season)]= stats['clean_sheet'][0]
        merged_players_df.at[index,'yellow_cards_{}'.format(season)]= stats['yellow_cards'][0]
        merged_players_df.at[index,'red_cards_{}'.format(season)]= stats['red_cards'][0]
    except:
        #print(id)
        n=n+1     
print('appearance,goal and card data merged')
print(merged_players_df.info());#print(merged_players_df.describe())

appearance,goal and card data merged
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28472 entries, 0 to 28471
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   player_id                             28472 non-null  int64  
 1   games_2022                            28472 non-null  float64
 2   minutes_played_2022                   28472 non-null  float64
 3   goals_2022                            28472 non-null  float64
 4   assists_2022                          28472 non-null  float64
 5   goals_against_2022                    28472 non-null  float64
 6   goals_for_2022                        28472 non-null  float64
 7   clean_sheet_2022                      28472 non-null  float64
 8   name                                  28472 non-null  object 
 9   position                              28472 non-null  object 
 10  sub_position                          28314 n

In [12]:
merged_players_df=merged_players_df.dropna(subset=['market_value_in_eur'])

df_league = pd.get_dummies(merged_players_df['current_club_domestic_competition_id'])
merged_players_df_new = pd.concat([merged_players_df, df_league], axis=1)
print(merged_players_df_new)
merged_players_df_new.corr()['market_value_in_eur']

       player_id  games_2022  minutes_played_2022  goals_2022  assists_2022  goals_against_2022  goals_for_2022  clean_sheet_2022                name    position        sub_position  last_season   foot  height_in_cm  age country_of_citizenship country_of_birth current_club_name club_value  squad_size current_club_domestic_competition_id    agent_name  market_value_in_eur  highest_market_value_in_eur  yellow_cards_2022  red_cards_2022  BE1  DK1  ES1  FR1  GB1  GR1  IT1  L1  NL1  PO1  RU1  SC1  TR1  UKR1
21          4742         9.0                728.0         0.0           0.0                17.0             7.0               0.0  Christophe Lepoint    Midfield    Central Midfield         2022  right         189.0   39                Belgium          Belgium       RFC Seraing        NaN        28.0                                  BE1   Eleven Plus              75000.0                    1700000.0                1.0             0.0    1    0    0    0    0    0    0   0    0    0    0 

player_id                      0.017250
games_2022                     0.448308
minutes_played_2022            0.453136
goals_2022                     0.457984
assists_2022                   0.432271
goals_against_2022             0.340250
goals_for_2022                 0.533702
clean_sheet_2022               0.491074
last_season                    0.214525
height_in_cm                  -0.005924
age                           -0.079592
squad_size                     0.056421
market_value_in_eur            1.000000
highest_market_value_in_eur    0.828984
yellow_cards_2022              0.306884
red_cards_2022                 0.060582
BE1                           -0.041867
DK1                           -0.053078
ES1                            0.057984
FR1                            0.038388
GB1                            0.244299
GR1                           -0.065483
IT1                            0.017579
L1                             0.062150
NL1                           -0.038097


In [13]:
merged_players_df1 = merged_players_df.copy()

#onehotvectpr lig
unique_leagues = pd.get_dummies(merged_players_df1[['current_club_domestic_competition_id']], prefix_sep='_')
merged_players_df1 = pd.concat([merged_players_df1, unique_leagues], axis=1) 


unique_positions = pd.get_dummies(merged_players_df1[['position']], prefix_sep='_')
merged_players_df1 = pd.concat([merged_players_df1, unique_positions], axis=1) 
# convert position categories to Columns for test data

unique_subpos=pd.get_dummies(merged_players_df1[['sub_position']], prefix_sep='_') #sub position
merged_players_df1 = pd.concat([merged_players_df1, unique_subpos], axis=1) 

unique_foot=pd.get_dummies(merged_players_df1[['foot']], prefix_sep='_') #foot
merged_players_df1 = pd.concat([merged_players_df1, unique_foot], axis=1) 
#print(merged_players_df1[NUMERIC_COLUMNS].head())

merged_players_df1

,player_id,games_2022,minutes_played_2022,goals_2022,assists_2022,goals_against_2022,goals_for_2022,clean_sheet_2022,name,position,sub_position,last_season,foot,height_in_cm,age,country_of_citizenship,country_of_birth,current_club_name,club_value,squad_size,current_club_domestic_competition_id,agent_name,market_value_in_eur,highest_market_value_in_eur,yellow_cards_2022,red_cards_2022,current_club_domestic_competition_id_BE1,current_club_domestic_competition_id_DK1,current_club_domestic_competition_id_ES1,current_club_domestic_competition_id_FR1,current_club_domestic_competition_id_GB1,current_club_domestic_competition_id_GR1,current_club_domestic_competition_id_IT1,current_club_domestic_competition_id_L1,current_club_domestic_competition_id_NL1,current_club_domestic_competition_id_PO1,current_club_domestic_competition_id_RU1,current_club_domestic_competition_id_SC1,current_club_domestic_competition_id_TR1,current_club_domestic_competition_id_UKR1,position_Attack,position_Defender,position_Goalkeeper,position_Midfield,position_Missing,sub_position_Attacking Midfield,sub_position_Central Midfield,sub_position_Centre-Back,sub_position_Centre-Forward,sub_position_Defensive Midfield,sub_position_Goalkeeper,sub_position_Left Midfield,sub_position_Left Winger,sub_position_Left-Back,sub_position_Right Midfield,sub_position_Right Winger,sub_position_Right-Back,sub_position_Second Striker,foot_both,foot_left,foot_right
21,4742,9.0,728.0,0.0,0.0,17.0,7.0,0.0,Christophe Lepoint,Midfield,Central Midfield,2022,right,189.0,39,Belgium,Belgium,RFC Seraing,NaN,28.0,BE1,Eleven Plus,75000.0,1700000.0,1.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
40,8187,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Juanmi Callejón,Attack,Left Winger,2012,right,180.0,36,Spain,Spain,APO Levadiakos,NaN,31.0,GR1,NaN,200000.0,1000000.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
51,10471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Luiz Gustavo,Midfield,Defensive Midfield,2021,left,187.0,36,Brazil,Brazil,Fenerbahce,NaN,28.0,TR1,ROGON,1000000.0,22000000.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
90,18940,4.0,330.0,0.0,0.0,4.0,9.0,0.0,Rémy Riou,Goalkeeper,Goalkeeper,2022,right,191.0,36,France,France,Olympique Lyon,NaN,23.0,FR1,NaN,400000.0,4000000.0,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
98,21042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Michal Kadlec,Defender,Centre-Back,2015,left,185.0,39,Czech Republic,CSSR,Fenerbahce,NaN,28.0,TR1,SPORT INVEST,200000.0,7500000.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28460,929600,5.0,213.0,2.0,0.0,6.0,7.0,1.0,Etim,Attack,Centre-Forward,2022,right,186.0,21,Nigeria,Nigeria,FC Vizela,NaN,26.0,PO1,10Management,300000.0,300000.0,1.0,1.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
28461,874764,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Mehmet Erdogan,Goalkeeper,Goalkeeper,2022,right,193.0,19,Turkey,NaN,Konyaspor,NaN,29.0,TR1,NaN,50000.0,50000.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
28466,188470,18.0,1580.0,3.0,0.0,15.0,26.0,9.0,Francisco Calvo,Defender,Centre-Back,2022,left,180.0,31,Costa Rica,Costa Rica,Konyaspor,NaN,29.0,TR1,Borja Couce,1500000.0,1700000.0,6.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
28469,221118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Patrick Friday Eze,Attack,Centre-Forward,2017,right,187.0,31,Nigeria,Nigeria,Konyaspor,NaN,29.0,TR1,NaN,200000.0,775000.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1


## Before export the dataframe, let's check the first model's accuracy

In [15]:
#separate numeric columns
LEAGUE_COLUMNS = ['current_club_domestic_competition_id_BE1','current_club_domestic_competition_id_DK1','current_club_domestic_competition_id_ES1','current_club_domestic_competition_id_FR1','current_club_domestic_competition_id_GB1','current_club_domestic_competition_id_GR1','current_club_domestic_competition_id_IT1','current_club_domestic_competition_id_L1','current_club_domestic_competition_id_NL1','current_club_domestic_competition_id_PO1','current_club_domestic_competition_id_RU1','current_club_domestic_competition_id_SC1','current_club_domestic_competition_id_TR1']
NUMERIC_COLUMNS=['goals_2022','goals_against_2022', 'goals_for_2022','games_2022','assists_2022','minutes_played_2022','age','height_in_cm','squad_size','position_Attack','position_Defender','position_Goalkeeper','position_Midfield','sub_position_Attacking Midfield','sub_position_Central Midfield','sub_position_Centre-Back','sub_position_Centre-Forward','sub_position_Defensive Midfield','sub_position_Left Midfield','sub_position_Left Winger','sub_position_Left-Back','sub_position_Right Midfield','sub_position_Right Winger','sub_position_Right-Back','sub_position_Second Striker','foot_left','foot_right','foot_both', 'current_club_domestic_competition_id_BE1','current_club_domestic_competition_id_DK1','current_club_domestic_competition_id_ES1','current_club_domestic_competition_id_FR1','current_club_domestic_competition_id_GB1','current_club_domestic_competition_id_GR1','current_club_domestic_competition_id_IT1','current_club_domestic_competition_id_L1','current_club_domestic_competition_id_NL1','current_club_domestic_competition_id_PO1','current_club_domestic_competition_id_RU1','current_club_domestic_competition_id_SC1','current_club_domestic_competition_id_TR1']

OTHER_COLUMNS=['last_season','country_of_citizenship','country_of_birth','current_club_name','current_club_domestic_competition_id','agent_name'] 
y=merged_players_df1['market_value_in_eur']
X = merged_players_df1[NUMERIC_COLUMNS].fillna(-1000)

#split test and training data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=21)

#model data
clf = GradientBoostingRegressor(random_state=0)
clf.fit(X_train, y_train)

#assess accurancy of model
print("Accuracy: {}".format(clf.score(X_test, y_test)))

Accuracy: 0.6760078618227743


In [16]:
df = pd.DataFrame(merged_players_df1)
df.to_csv('merged_players_df_final.csv', index=False)